In [ ]:
import pandas as pd
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import pandas as pd
import jieba

# 🔹 1. 读取数据
data = pd.read_excel(r"二手汽车口碑.xlsx")

# 🔹 2. 指定你需要处理的列
columns_to_process = ['空间', '驾驶感受', '操控', '续航', '舒适性', '外观', '内饰', '性价比', '智能化']

# 🔹 3. 读取停用词和汽车词汇
with open(r"stopped_words.txt", 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

with open(r"汽车词汇词库.txt", 'r', encoding='utf-8') as f:
    car_terms = set(f.read().splitlines())

# 🔹 4. 文本清洗函数：拼接多个维度评论 -> 分词 -> 去停用词 -> 筛选汽车词汇/长度>1
def clean_and_tokenize_row(row):
    combined_text = ' '.join(str(row[col]) for col in columns_to_process if pd.notna(row[col]))
    words = jieba.lcut(combined_text)
    words = [word for word in words if word not in stopwords]
    words = [word for word in words if word in car_terms or len(word) > 1]
    return ' '.join(words)

# 🔹 5. 应用处理函数，生成新列
data['Cleaned_Comment'] = data.apply(clean_and_tokenize_row, axis=1)

# 🔹 6. 查看前几行结果
print(data[['Cleaned_Comment']].head())
# 🔹 7. 覆盖原文件
data.to_excel(r"二手汽车口碑.xlsx", index=False)


                                     Cleaned_Comment
0  车长 接近 5m 空间 前排 空间 很大 后排 腿部 空间 设计 头顶 空间 偏小 185 ...
1  空间 算是 劣势 毕竟 车子 整体 难搞 第二排 储物 空间 只有 后备箱 前面 挤出来 前...
2  这辆 内部空间 其实 外部 尺寸 比例 这么 这辆 长度 达到 米左右 车内 腿部 空间 想...
3  很多 后排 空间 实际上 不小 除非 一米 八五 身高 足够 起步 很快 方向 灵敏 好开 ...
4  内部 空间设计 其实 不错 车内 储物 能力 中控 扶手 下面 还有 空旷 位置 前备 后排...


In [ ]:
# 🔹 4. 划分数据集
X = data['Cleaned_Comment']
y = data['Sentiment']
# 这是它有一部分自带的标签，但是其他的没有，所以需要再预测

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🔹 5. 特征提取：TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# 假设 X_test 是一个包含 clean_comment 的 Series 或 DataFrame
# 如果你之前使用的是 Series，可以这样转成 DataFrame，并命名列
X_test_df = X_test.to_frame(name='clean_comment')

# 训练模型
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# 预测
y_pred = model.predict(X_test_tfidf)
y_pred_proba = model.predict_proba(X_test_tfidf)

# 构建包含 clean_comment 和预测信息的 DataFrame
results = X_test_df.copy()
results['Predicted'] = y_pred
results['Positive_Score'] = y_pred_proba[:, 1]
results['Negative_Score'] = y_pred_proba[:, 0]

# 保存为 Excel 文件
output_path = '简化情感预测结果.xlsx'
results.to_excel(output_path, index=False)

# 打印前 5 行
print("前5条预测结果如下：")
print(results.head())


前5条预测结果如下：
                                          clean_comment     Predicted  \
8188  台车 空间 很大 毕竟 中型 SUV 宽敞 车子 动力 充足 加速 很快 避震 效果 隔音 ...  non-positive   
298   空间 很大 后备箱 很能 后排 座椅 空间设计 利用率 还有 前备 使用 频率 车头 驾驶 ...  non-positive   
9140  空间 第三排 座椅 放平 放平 后备箱 空间 扩展 能装 第三排 空间 相对 两排 空间 一...      positive   
3199  毕竟 轴距 车身 尺寸 摆在 那里 空间 充裕 保证 每个 舒适 乘坐 空间 前排 驾驶 空...      positive   
1572  乘坐 空间 不错 这么 身材 驾驶座 宽宽松松 后排 体验 身材 坐三人 有点 拥挤 乘坐 ...      positive   

      Positive_Score  Negative_Score  
8188        0.451032        0.548968  
298         0.449290        0.550710  
9140        0.702258        0.297742  
3199        0.614279        0.385721  
1572        0.532724        0.467276  
